In [29]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# get data
from eden.converter.graph.gspan import gspan_to_eden
from itertools import islice
def get_graphs(dataset_fname, size=100):
    return  islice(gspan_to_eden(dataset_fname),size)

dataset_names = !cat NCI60/names
dataset = dataset_names[3]
# dataset_fname = 'bursi.pos.gspan'
dataset_fname = 'NCI60/' + dataset + '_orig_pos.gspan'
print 'Working with dataset: %s' % dataset 

Working with dataset: HOP_62_t


In [31]:
%%time
# induce a grammar and train an estimator
training_graphs = get_graphs(dataset_fname, size=1000)

from graphlearn.graphlearn import GraphLearnSampler
sampler=GraphLearnSampler(radius_list=[0,1], thickness_list=[1],
                          min_cip_count=2, min_interface_count=2)

sampler.fit(training_graphs, n_jobs=-1)

print('graph grammar stats:')
interface_counts, core_counts, cip_counts = sampler.grammar().size()
print('#interfaces: %d   #cores: %d   #core-interface-pairs: %d' % (interface_counts, core_counts, cip_counts))

graph grammar stats:
#interfaces: 221   #cores: 178   #core-interface-pairs: 1345
CPU times: user 1min 6s, sys: 6.71 s, total: 1min 12s
Wall time: 1min 17s


In [35]:
%%time
# graph sampling
graphs = get_graphs(dataset_fname, size=100)
id_start=1
id_end=id_start+12
from itertools import islice
graphs = islice(graphs,id_start,id_end)

# sample
sampler.feasibility_checker.draw_problem=False
graphs = sampler.sample(graphs,
                        n_steps=120, n_samples=6,
                        probabilistic_core_choice=True,
                        same_radius=False, same_core_size=True,
                        accept_annealing_factor=1.4,
                        n_jobs=-1,burnin=0,select_cip_max_tries=400,keep_duplicates=True)

CPU times: user 18 µs, sys: 10 µs, total: 28 µs
Wall time: 27.9 µs


In [33]:
%%time
# plot examples of sampling paths
from graphlearn.utils.draw import draw_graph_set_graphlearn, get_score_of_graph
scores=[]
ids=range(id_start,id_end)
for i,graph in enumerate(graphs):
    print 'Graph id: %d'%(ids[i])
    scores.append(graph.graph['sampling_info']['score_history'])
    path_graphs = graph.graph['sampling_info']['graphs_history']
    draw_graph_set_graphlearn(path_graphs,
                   n_graphs_per_line=6, size=7, 
                   colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='_labels_',
                   vertex_alpha=0.5, edge_alpha=0.2, node_size=250,
                   headlinehook=get_score_of_graph)

CPU times: user 10.6 ms, sys: 26.3 ms, total: 36.9 ms
Wall time: 150 ms


In [34]:
%%time
# plot sampling path score
from itertools import islice
import numpy as np
import pylab as plt
step=1
num_graphs_per_plot=3
num_plots=np.ceil([len(scores)/num_graphs_per_plot])
for i in range(num_plots):
    plt.figure(figsize=(10,5))
    for j,score in enumerate(scores[i*num_graphs_per_plot:i*num_graphs_per_plot+num_graphs_per_plot]):
        data = list(islice(score,None, None, step))
        plt.plot(data, linewidth=2, label='graph %d'%(j+i*num_graphs_per_plot+id_start))
        plt.plot(data, linestyle='None', markerfacecolor='white', marker='o', markeredgewidth=1,markersize=5)
    plt.legend(loc='lower right')
    plt.grid()
    plt.ylim(-0.1,1.1)
    plt.show()

CPU times: user 217 µs, sys: 261 µs, total: 478 µs
Wall time: 410 µs


.